In [ ]:
import json
import os
import random

In [ ]:
random.seed(0)

In [ ]:
result_dic = {}

In [ ]:
data_path = "./English"

with open(os.path.join(data_path, "data.json"), "r", encoding="utf-8") as f:
    data = json.load(f)
f = open(os.path.join(data_path, "valListFile.txt"))
val_list = f.read().splitlines()
f.close()
f = open(os.path.join(data_path, "testListFile.txt"))
test_list = f.read().splitlines()
f.close()
train_list = list(filter(lambda x: x not in test_list + val_list, data.keys()))


In [ ]:

split_id_dic = {
    "train": [],
    "val": [],
    "test": [],
}


split_dial_dic = {
    "train": {},
    "val": {},
    "test": {},
}


for dial_id, dial in data.items():
    if dial_id in test_list:
        split_id_dic["test"].append(dial_id)
        split_dial_dic["test"][dial_id] = dial
    elif dial_id in val_list:
        split_id_dic["val"].append(dial_id)
        split_dial_dic["val"][dial_id] = dial

    elif dial_id in train_list:
        split_id_dic["train"].append(dial_id)
        split_dial_dic["train"][dial_id] = dial


In [ ]:
# Get the set of dialogue id that maximise the trigram coverage of the chosen dialogues.

In [ ]:

def get_trigram_of_dialogue(dial):
    all_trigram_set = set()
    for item in (dial['log']):
        text = item["text"].lower()
        tokens = text.split()
        temp_trigram_list = ([(a, b, c) for a, b, c in zip(tokens, tokens[1:], tokens[2:])])
        all_trigram_set.update(temp_trigram_list)
    return all_trigram_set


dialID_trigram_dic = {}    
all_avaliable_dialID = []


for dial_id, dial in split_dial_dic["train"].items():
    dial_trigram = get_trigram_of_dialogue(dial)
    all_avaliable_dialID.append(dial_id)
    dialID_trigram_dic[dial_id] = dial_trigram



In [ ]:
def Sort(sub_li):
   
    l = len(sub_li)
     
    for i in range(0, l):
        for j in range(0, l-i-1):
             
            if (len(sub_li[j][1]) < len(sub_li[j + 1][1])):
                tempo = sub_li[j]
                sub_li[j] = sub_li[j + 1]
                sub_li[j + 1] = tempo
     
    return sub_li


In [ ]:
added_trigran_set = set()

dialID_trigrams_list = []

for dial_id, trigrams in dialID_trigram_dic.items():
    dialID_trigrams_list.append((dial_id, trigrams))

In [ ]:
# This code will take a while.
ans_dial_id_list = []

for i in range(len(dialID_trigram_dic)):
    if i % 100 == 0:
        print(i)
    dialID_trigrams_list = list(map(lambda x : (x[0], x[1] - added_trigran_set), dialID_trigrams_list))
    dialID_trigrams_list = (Sort(dialID_trigrams_list))
    
    this_element = dialID_trigrams_list[0]
    dialID_trigrams_list = dialID_trigrams_list[1:]
    added_trigran_set.update(this_element[1])
    ans_dial_id_list.append(this_element[0])

In [ ]:
len(ans_dial_id_list)

In [ ]:

for i in range(0, 101):
    if i > 10 and i%10 > 0:
        continue
    proportion = (i / 100)
    idx = int(len(ans_dial_id_list) * proportion)
    target_list = ans_dial_id_list[:idx]
    result_dic["train"]["trigram" + str(proportion)] = target_list

In [ ]:
def get_trigram_of_dialogue(dial):
    all_trigram_set = set()
    for item in (dial['log']):
        text = item["text"].lower()
        tokens = text.split()
        temp_trigram_list = ([(a, b, c) for a, b, c in zip(tokens, tokens[1:], tokens[2:])])
        all_trigram_set.update(temp_trigram_list)
    return all_trigram_set


dialID_trigram_dic = {}    
all_avaliable_dialID = []


for dial_id, dial in split_dial_dic["val"].items():
    dial_trigram = get_trigram_of_dialogue(dial)
    all_avaliable_dialID.append(dial_id)
    dialID_trigram_dic[dial_id] = dial_trigram



In [ ]:
added_trigran_set = set()

dialID_trigrams_list = []

for dial_id, trigrams in dialID_trigram_dic.items():
    dialID_trigrams_list.append((dial_id, trigrams))
    



In [ ]:
ans_dial_id_list = []

for i in range(len(dialID_trigram_dic)):
    if i % 100 == 0:
        print(i)
    dialID_trigrams_list = list(map(lambda x : (x[0], x[1] - added_trigran_set), dialID_trigrams_list))
    dialID_trigrams_list = (Sort(dialID_trigrams_list))
    
    this_element = dialID_trigrams_list[0]
    dialID_trigrams_list = dialID_trigrams_list[1:]
    
    added_trigran_set.update(this_element[1])
    ans_dial_id_list.append(this_element[0])

In [ ]:
for i in range(0, 101):
    if i > 10 and i%10 > 0:
        continue
    proportion = (i / 100)
    idx = int(len(ans_dial_id_list) * proportion)
    target_list = ans_dial_id_list[:idx]
    result_dic["val"]["trigram" + str(proportion)] = target_list

In [ ]:
# Get the set of dialogue id that maximise the domain coverage of the chosen dialogues.

In [ ]:
# 
def get_domain_of_dialogue(dial):
    all_domain_set = set()
    for item in (dial['log']):
        for domain_intent in item["dialog_act"].keys():
            all_domain_set.add(domain_intent.split("-")[0])
    if "general" in all_domain_set:
        all_domain_set.remove("general")
    if "Booking" in all_domain_set:
        all_domain_set.remove("Booking")
        
    return all_domain_set

dialID_domain_dic = {}    
domain_id_dic = {}
all_avaliable_dialID = []


for dial_id, dial in split_dial_dic["train"].items():
    dial_domain = get_domain_of_dialogue(dial)
    all_avaliable_dialID.append(dial_id)
    dialID_domain_dic[dial_id] = dial_domain
    for item in list(dial_domain):
        temp_set = domain_id_dic.get(item, set())
        temp_set.add(dial_id)
        domain_id_dic[item] = temp_set
# get_domain_of_dialogue(data["SNG01856.json"])


In [ ]:
all_domain_list = list(domain_id_dic.keys())
answer_dial_id_seq = []


for i in range(100000):
    current_domain =  all_domain_list[i%len(all_domain_list)]
    
    ans = None
    all_avalaible_id_set = domain_id_dic[current_domain]
    if len(all_avalaible_id_set) <= 0:
        continue
    item = random.sample(all_avalaible_id_set, 1)[0]
    all_avalaible_id_set.remove(item)
    answer_dial_id_seq.append(item)
    
    for domain, temp_set in domain_id_dic.items():
        if item in temp_set:
            temp_set.remove(item)
    assert item in all_avaliable_dialID
    all_avaliable_dialID.remove(item)
    
    if len(all_avaliable_dialID) <= 0:
        break

In [ ]:

for i in range(0, 101):
    if i > 10 and i%10 > 0:
        continue
    proportion = (i / 100)
    idx = int(len(answer_dial_id_seq) * proportion)
    target_list = answer_dial_id_seq[:idx]
    result_dic["train"]["domain" + str(proportion)] = target_list

In [ ]:
def get_domain_of_dialogue(dial):
    all_domain_set = set()
    for item in (dial['log']):
        for domain_intent in item["dialog_act"].keys():
            all_domain_set.add(domain_intent.split("-")[0])
    if "general" in all_domain_set:
        all_domain_set.remove("general")
    if "Booking" in all_domain_set:
        all_domain_set.remove("Booking")
        
    return all_domain_set

dialID_domain_dic = {}    
domain_id_dic = {}
all_avaliable_dialID = []


for dial_id, dial in split_dial_dic["val"].items():
    dial_domain = get_domain_of_dialogue(dial)
    all_avaliable_dialID.append(dial_id)
    dialID_domain_dic[dial_id] = dial_domain
    for item in list(dial_domain):
        temp_set = domain_id_dic.get(item, set())
        temp_set.add(dial_id)
        domain_id_dic[item] = temp_set
# get_domain_of_dialogue(data["SNG01856.json"])


In [ ]:
all_domain_list = list(domain_id_dic.keys())
answer_dial_id_seq = []


for i in range(100000):
    current_domain =  all_domain_list[i%len(all_domain_list)]
    
    ans = None
    all_avalaible_id_set = domain_id_dic[current_domain]
    if len(all_avalaible_id_set) <= 0:
        continue
    item = random.sample(all_avalaible_id_set, 1)[0]
    all_avalaible_id_set.remove(item)
    answer_dial_id_seq.append(item)
    
    for domain, temp_set in domain_id_dic.items():
        if item in temp_set:
            temp_set.remove(item)
    assert item in all_avaliable_dialID
    all_avaliable_dialID.remove(item)
    
    if len(all_avaliable_dialID) <= 0:
        break

In [ ]:
len(answer_dial_id_seq)

In [ ]:
len(all_avaliable_dialID)

In [ ]:

for i in range(0, 101):
    if i > 10 and i%10 > 0:
        continue
    proportion = (i / 100)
    idx = int(len(answer_dial_id_seq) * proportion)
    target_list = answer_dial_id_seq[:idx]
    result_dic["val"]["domain" + str(proportion)] = target_list

In [ ]:
# Get the set of dialogue id that maximise the slot coverage of the chosen dialogues.

In [ ]:
def get_slot_of_dialogue(dial):
    all_slot_set = set()
    for item in (dial['log']):
        for domain_intent in item["dialog_act"].keys():
            for s_v_pair in (item["dialog_act"][domain_intent]):
                all_slot_set.add(s_v_pair[0])
    return all_slot_set


dialID_slot_dic = {}    
slot_id_dic = {}
all_avaliable_dialID = []


for dial_id, dial in split_dial_dic["train"].items():
    dial_slot = get_slot_of_dialogue(dial)
    all_avaliable_dialID.append(dial_id)
    dialID_slot_dic[dial_id] = dial_slot
    
    for item in list(dial_slot):
        temp_set = slot_id_dic.get(item, set())
        temp_set.add(dial_id)
        slot_id_dic[item] = temp_set


In [ ]:
all_slot_list = list(slot_id_dic.keys())
answer_dial_id_seq = []


for i in range(100000):
    current_slot =  all_slot_list[i%len(all_slot_list)]
    
    ans = None
    all_avalaible_id_set = slot_id_dic[current_slot]
    if len(all_avalaible_id_set) <= 0:
        continue
    item = random.sample(all_avalaible_id_set, 1)[0]
    all_avalaible_id_set.remove(item)
    answer_dial_id_seq.append(item)
    
    for slot, temp_set in slot_id_dic.items():
        if item in temp_set:
            temp_set.remove(item)
    assert item in all_avaliable_dialID
    all_avaliable_dialID.remove(item)
    
    if len(all_avaliable_dialID) <= 0:
        break


In [ ]:
for i in range(0, 101):
    if i > 10 and i%10 > 0:
        continue
    proportion = (i / 100)
    idx = int(len(answer_dial_id_seq) * proportion)
    target_list = answer_dial_id_seq[:idx]
    result_dic["train"]["slot" + str(proportion)] = target_list

In [ ]:
def get_slot_of_dialogue(dial):
    all_slot_set = set()
    for item in (dial['log']):
        for domain_intent in item["dialog_act"].keys():
            for s_v_pair in (item["dialog_act"][domain_intent]):
                all_slot_set.add(s_v_pair[0])
    return all_slot_set


dialID_slot_dic = {}    
slot_id_dic = {}
all_avaliable_dialID = []


for dial_id, dial in split_dial_dic["val"].items():
    dial_slot = get_slot_of_dialogue(dial)
    all_avaliable_dialID.append(dial_id)
    dialID_slot_dic[dial_id] = dial_slot
    
    for item in list(dial_slot):
        temp_set = slot_id_dic.get(item, set())
        temp_set.add(dial_id)
        slot_id_dic[item] = temp_set


In [ ]:
all_slot_list = list(slot_id_dic.keys())
answer_dial_id_seq = []


for i in range(100000):
    current_slot =  all_slot_list[i%len(all_slot_list)]
    
    ans = None
    all_avalaible_id_set = slot_id_dic[current_slot]
    if len(all_avalaible_id_set) <= 0:
        continue
    item = random.sample(all_avalaible_id_set, 1)[0]
    all_avalaible_id_set.remove(item)
    answer_dial_id_seq.append(item)
    
    for slot, temp_set in slot_id_dic.items():
        if item in temp_set:
            temp_set.remove(item)
    assert item in all_avaliable_dialID
    all_avaliable_dialID.remove(item)
    
    if len(all_avaliable_dialID) <= 0:
        break
    

In [ ]:
len(answer_dial_id_seq)

In [ ]:
for i in range(0, 101):
    if i > 10 and i%10 > 0:
        continue
    proportion = (i / 100)
    idx = int(len(answer_dial_id_seq) * proportion)
    target_list = answer_dial_id_seq[:idx]
    result_dic["val"]["slot" + str(proportion)] = target_list

In [ ]:
# Get the set of dialogue id that maximise the length of the choosen dialogues.

In [ ]:
dial_len_list = []

for dial_id, dial in split_dial_dic["train"].items():
    dial_len = len(dial["log"])
    

    dial_len_list.append((dial_id,dial_len))
    
def Sort(sub_li):
   
    l = len(sub_li)
     
    for i in range(0, l):
        for j in range(0, l-i-1):
             
            if (sub_li[j][1] > sub_li[j + 1][1]):
                tempo = sub_li[j]
                sub_li[j] = sub_li[j + 1]
                sub_li[j + 1] = tempo
     
    return sub_li
answer_dial_id_seq = list(map(lambda x : x[0], reversed(Sort(dial_len_list))))

In [ ]:
for i in range(0, 101):
    if i > 10 and i%10 > 0:
        continue
    proportion = (i / 100)
    idx = int(len(answer_dial_id_seq) * proportion)
    target_list = answer_dial_id_seq[:idx]
    result_dic["train"]["length" + str(proportion)] = target_list

In [ ]:
dial_len_list = []

for dial_id, dial in split_dial_dic["val"].items():
    dial_len = len(dial["log"])
    

    dial_len_list.append((dial_id,dial_len))
    
def Sort(sub_li):
   
    l = len(sub_li)
     
    for i in range(0, l):
        for j in range(0, l-i-1):
             
            if (sub_li[j][1] > sub_li[j + 1][1]):
                tempo = sub_li[j]
                sub_li[j] = sub_li[j + 1]
                sub_li[j + 1] = tempo
     
    return sub_li
answer_dial_id_seq = list(map(lambda x : x[0], reversed(Sort(dial_len_list))))

In [ ]:
len(answer_dial_id_seq)

In [ ]:
for i in range(0, 101):
    if i > 10 and i%10 > 0:
        continue
    proportion = (i / 100)
    idx = int(len(answer_dial_id_seq) * proportion)
    target_list = answer_dial_id_seq[:idx]
    result_dic["val"]["lentgh" + str(proportion)] = target_list

In [ ]:
# Get random split of the dialogues

In [ ]:

for i in range(0, 101):
    if i > 10 and i%10 > 0:
        continue
    proportion = (i / 100)
    target_list = random.sample(split_id_dic["train"], int(len(split_id_dic["train"]) * proportion))
    result_dic["train"]["random" + str(proportion)] = target_list

In [ ]:
for i in range(0, 101):
    if i > 10 and i%10 > 0:
        continue
    proportion = (i / 100)
    target_list = random.sample(split_id_dic["val"], int(len(split_id_dic["val"]) * proportion))
    result_dic["val"]["random" + str(proportion)] = target_list

In [ ]:
for i in range(0, 101):
    if i > 10 and i%10 > 0:
        continue
    proportion = (i / 100)
    target_list = random.sample(split_id_dic["test"], int(len(split_id_dic["test"]) * proportion))
    result_dic["test"]["random" + str(proportion)] = target_list

In [ ]:
with open('./parallel_dic.json', 'w', encoding='utf-8') as f:
    json.dump(result_dic, f, ensure_ascii=False)